In [1]:
import warnings
warnings.filterwarnings('ignore')
import qlib
from qlib.config import REG_CN
from qlib.contrib.data.handler import Alpha158

data_handler_config = {
    "start_time": "2018-01-01",
    "end_time": "2021-06-03",
    "fit_start_time": "2019-01-01",
    "fit_end_time": "2020-12-31",
    "instruments": "csi100",
}

if __name__ == "__main__":
    provider_uri = "~/.qlib/qlib_data/qlib_cn_1d"  # target_dir
    qlib.init(provider_uri=provider_uri, region=REG_CN)
    h = Alpha158(**data_handler_config)

    # get all the columns of the data
    print(h.get_cols())

    # fetch all the labels
    print(h.fetch(col_set="label"))

    # fetch all the features
    print(h.fetch(col_set="feature"))

[13171:MainThread](2021-06-04 00:07:23,511) INFO - qlib.Initialization - [config.py:284] - default_conf: client.
[13171:MainThread](2021-06-04 00:07:25,118) INFO - qlib.Initialization - [__init__.py:48] - qlib successfully initialized based on client settings.
[13171:MainThread](2021-06-04 00:07:25,120) INFO - qlib.Initialization - [__init__.py:49] - data_path=/Users/harry/.qlib/qlib_data/qlib_cn_1d


['KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT', 'KSFT2', 'OPEN0', 'HIGH0', 'LOW0', 'VWAP0', 'ROC5', 'ROC10', 'ROC20', 'ROC30', 'ROC60', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'STD5', 'STD10', 'STD20', 'STD30', 'STD60', 'BETA5', 'BETA10', 'BETA20', 'BETA30', 'BETA60', 'RSQR5', 'RSQR10', 'RSQR20', 'RSQR30', 'RSQR60', 'RESI5', 'RESI10', 'RESI20', 'RESI30', 'RESI60', 'MAX5', 'MAX10', 'MAX20', 'MAX30', 'MAX60', 'MIN5', 'MIN10', 'MIN20', 'MIN30', 'MIN60', 'QTLU5', 'QTLU10', 'QTLU20', 'QTLU30', 'QTLU60', 'QTLD5', 'QTLD10', 'QTLD20', 'QTLD30', 'QTLD60', 'RANK5', 'RANK10', 'RANK20', 'RANK30', 'RANK60', 'RSV5', 'RSV10', 'RSV20', 'RSV30', 'RSV60', 'IMAX5', 'IMAX10', 'IMAX20', 'IMAX30', 'IMAX60', 'IMIN5', 'IMIN10', 'IMIN20', 'IMIN30', 'IMIN60', 'IMXD5', 'IMXD10', 'IMXD20', 'IMXD30', 'IMXD60', 'CORR5', 'CORR10', 'CORR20', 'CORR30', 'CORR60', 'CORD5', 'CORD10', 'CORD20', 'CORD30', 'CORD60', 'CNTP5', 'CNTP10', 'CNTP20', 'CNTP30', 'CNTP60', 'CNTN5', 'CNTN10', 'CNTN20', 'CNTN30', 'CNT

In [6]:
from qlib.data import D
instruments = ['sz300643']
fields = ['$close', '$volume', 'Ref($close, 1)', 'Mean($close, 3)', '$high-$low']
D.features(instruments, fields, start_time='2020-01-01', end_time='2021-06-03', freq='day').tail()

$close     $volume  Ref($close, 1)  Mean($close, 3)  \
instrument datetime                                                            
sz300643   2021-05-28  1.867190  39383888.0        1.811257         1.827159   
           2021-05-31  1.880351  30428524.0        1.867190         1.852933   
           2021-06-01  1.852384  23077452.0        1.880351         1.866642   
           2021-06-02  1.776710  30782904.0        1.852384         1.836482   
           2021-06-03  1.778355  15098765.0        1.776710         1.802483   

                       $high-$low  
instrument datetime                
sz300643   2021-05-28    0.055933  
           2021-05-31    0.047708  
           2021-06-01    0.042773  
           2021-06-02    0.093771  
           2021-06-03    0.024676

In [13]:
data = h.fetch(col_set=["feature","label"])

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args


# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]

                train_array = np.sort(np.unique(
                    np.concatenate((train_array,
                                    train_array_tmp)),
                    axis=None), axis=None)

            train_end = train_array.size

            for test_group_idx in unique_groups[group_test_start:
                    group_test_start +
                    group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                    np.concatenate((test_array,
                                    test_array_tmp)),
                    axis=None), axis=None)

            test_array = test_array[group_gap:]

            if self.verbose > 0:
                pass

            yield [int(i) for i in train_array], [int(i) for i in test_array]


In [4]:
import numpy as np
import pandas as pd
import pickle
from typing import Text, Union
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

def create_ae_mlp(num_columns, num_labels, hidden_units, dropout_rates, ls=1e-2, lr=1e-3):
    inp = tf.keras.layers.Input(shape=(num_columns,))
    x0 = tf.keras.layers.BatchNormalization()(inp)

    encoder = tf.keras.layers.GaussianNoise(dropout_rates[0])(x0)
    encoder = tf.keras.layers.Dense(hidden_units[0])(encoder)
    encoder = tf.keras.layers.BatchNormalization()(encoder)
    encoder = tf.keras.layers.Activation('swish')(encoder)

    decoder = tf.keras.layers.Dropout(dropout_rates[1])(encoder)
    decoder = tf.keras.layers.Dense(num_columns, name='decoder')(decoder)

    x_ae = tf.keras.layers.Dense(hidden_units[1])(decoder)
    x_ae = tf.keras.layers.BatchNormalization()(x_ae)
    x_ae = tf.keras.layers.Activation('swish')(x_ae)
    x_ae = tf.keras.layers.Dropout(dropout_rates[2])(x_ae)

    out_ae = tf.keras.layers.Dense(num_labels, activation='linear', name='ae_action')(x_ae)

    x = tf.keras.layers.Concatenate()([x0, encoder])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rates[3])(x)

    for i in range(2, len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('swish')(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 2])(x)

    out = tf.keras.layers.Dense(num_labels, activation='linear', name='action')(x)

    model = tf.keras.models.Model(inputs=inp, outputs=[decoder, out_ae, out])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss={'decoder': tf.keras.losses.MeanSquaredError(),
                        'ae_action': tf.keras.losses.MeanSquaredError(),
                        'action': tf.keras.losses.MeanSquaredError(),
                        },
                  metrics={'decoder': tf.keras.metrics.MeanAbsoluteError(name='mae'),
                           'ae_action': tf.keras.metrics.MeanAbsoluteError(name='mae'),
                           'action': tf.keras.metrics.MeanAbsoluteError(name='mae'),
                           },
                  )

    return model

In [15]:
X, y = data["feature"].values, data["label"].values
num_columns = X.shape[1]
num_labels = 1
params = {'num_columns': num_columns,
          'num_labels': num_labels,
          'hidden_units': [96, 96, 896, 448, 448, 256],
          'dropout_rates': [0.03527936123679956, 0.038424974585075086, 0.42409238408801436, 0.10431484318345882,
                            0.49230389137187497, 0.32024444956111164, 0.2716856145683449, 0.4379233941604448],
          'ls': 0,
          'lr': 1e-3,
          }
group_index = data["feature"].index.get_level_values('datetime').astype(str)
gkf = PurgedGroupTimeSeriesSplit(n_splits=5, group_gap=2)
for fold, (tr, te) in enumerate(gkf.split(X, y, group_index)):
    model = create_ae_mlp(**params)
    es = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=2, mode='max',
                       baseline=None, restore_best_weights=True, verbose=1)
    model.fit(X[tr], [X[tr], y[tr], y[tr]],
              validation_data=(X[te], [X[te], y[te], y[te]]), epochs=10,
              batch_size=4096, verbose=1)
    model.save(f'model_{fold}.h5')

Epoch 1/10
4/4 [==============================] - 6s 723ms/step - loss: nan - decoder_loss: nan - ae_action_loss: nan - action_loss: nan - decoder_mae: nan - ae_action_mae: nan - action_mae: nan - val_loss: nan - val_decoder_loss: nan - val_ae_action_loss: nan - val_action_loss: nan - val_decoder_mae: nan - val_ae_action_mae: nan - val_action_mae: nan
Epoch 2/10
4/4 [==============================] - 2s 561ms/step - loss: nan - decoder_loss: nan - ae_action_loss: nan - action_loss: nan - decoder_mae: nan - ae_action_mae: nan - action_mae: nan - val_loss: nan - val_decoder_loss: nan - val_ae_action_loss: nan - val_action_loss: nan - val_decoder_mae: nan - val_ae_action_mae: nan - val_action_mae: nan
Epoch 3/10
4/4 [==============================] - 2s 619ms/step - loss: nan - decoder_loss: nan - ae_action_loss: nan - action_loss: nan - decoder_mae: nan - ae_action_mae: nan - action_mae: nan - val_loss: nan - val_decoder_loss: nan - val_ae_action_loss: nan - val_action_loss: nan - val_de

In [13]:
def set_tag(*args, **kwargs):
    params = {}
    params.update(kwargs)
    return params

In [14]:
set_tag(**{'a':'b'})

{'a': 'b'}

In [41]:
import os
import yaml

def scanf_tasks(config_root_path):
    tasks = []
    for file in os.listdir(config_root_path):
        if not file.endswith(".yaml"):
                continue
        with open(os.path.join(config_root_path, file)) as fp:
            config = yaml.safe_load(fp)
            tasks.append(config.get("task"))

    return tasks

In [42]:
config_root_path = "/Users/harry/Workspace/qlib/examples/benchmarks/MLP"
tasks = scanf_tasks(config_root_path)

In [30]:
tasks[0]

{'model': {'class': 'GRU',
  'module_path': 'qlib.contrib.model.pytorch_gru_ts',
  'kwargs': {'d_feat': 20,
   'hidden_size': 64,
   'num_layers': 2,
   'dropout': 0.0,
   'n_epochs': 200,
   'lr': '2e-4',
   'early_stop': 10,
   'batch_size': 800,
   'metric': 'loss',
   'loss': 'mse',
   'n_jobs': 20,
   'GPU': 0}},
 'dataset': {'class': 'TSDatasetH',
  'module_path': 'qlib.data.dataset',
  'kwargs': {'handler': {'class': 'Alpha158',
    'module_path': 'qlib.contrib.data.handler',
    'kwargs': {'start_time': datetime.date(2014, 6, 1),
     'end_time': datetime.date(2021, 6, 3),
     'fit_start_time': datetime.date(2014, 6, 1),
     'fit_end_time': datetime.date(2020, 12, 31),
     'instruments': 'csi300',
     'infer_processors': [{'class': 'FilterCol',
       'kwargs': {'fields_group': 'feature',
        'col_list': ['RESI5',
         'WVMA5',
         'RSQR5',
         'KLEN',
         'RSQR10',
         'CORR5',
         'CORD5',
         'CORR10',
         'ROC60',
         'RES